<a href="https://colab.research.google.com/github/Damilola-ops/Text-de-shuffling/blob/main/Text_de_shuffling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets
!pip install pandas 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.1 MB/s 
     |████████████████████████████████| 362 kB 51.4 MB/s 
     |████████████████████████████████| 101 kB 9.2 MB/s 
     |████████████████████████████████| 6.6 MB 45.6 MB/s 
     |████████████████████████████████| 596 kB 34.6 MB/s 
     |████████████████████████████████| 212 kB 55.1 MB/s 
     |████████████████████████████████| 1.1 MB 36.8 MB/s 
     |████████████████████████████████| 140 kB 53.9 MB/s 
     |████████████████████████████████| 127 kB 11.0 MB/s 
     |████████████████████████████████| 144 kB 28.3 MB/s 
     |████████████████████████████████| 271 kB 59.4 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing 

In [ ]:
from transformers import EncoderDecoderModel, EncoderDecoderConfig, BertTokenizerFast, Seq2SeqTrainingArguments, Seq2SeqTrainer, BertConfig
import torch
import pandas as pd 
import numpy as np
import sklearn.preprocessing 
from torch.nn import functional as F 
import datasets
from datasets import load_dataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab datasets/Text-Deshuffling/val (1).csv',index_col=[0])
test_df = pd.read_csv('/content/drive/MyDrive/Colab datasets/Text-Deshuffling/test (9).csv',index_col=[0])

val_df = pd.read_csv('/content/drive/MyDrive/Colab datasets/Text-Deshuffling/val (1).csv')

In [ ]:
#using huggingface's dataset function to load multiple datasets at once 
dataset = load_dataset( 'csv',data_files = {'train':['/content/drive/MyDrive/Colab datasets/Text-Deshuffling/val (1).csv'],
                                         'val':['/content/drive/MyDrive/Colab datasets/Text-Deshuffling/test (9).csv'],
                                         'test':['/content/drive/MyDrive/Colab datasets/Text-Deshuffling/val (1).csv']})

Using custom data configuration default-6663dddeb455b2d4


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-6663dddeb455b2d4/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'id'],
        num_rows: 4001
    })
    val: Dataset({
        features: ['text', 'label', 'id'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['text', 'label', 'id'],
        num_rows: 4001
    })
})

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokenizer

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
sample = train_df['text'].loc[1]
tokens = tokenizer.tokenize(sample)
token_id = tokenizer.convert_tokens_to_ids(tokens)
print('sample text: ', sample)
print('tokens : ',tokens)
print('token_id : ',token_id)

sample text:  approaches achieved networks Recent neural im-pressive performance. with deep have convolutional
tokens :  ['approaches', 'achieved', 'networks', 'recent', 'neural', 'im', '-', 'press', '##ive', 'performance', '.', 'with', 'deep', 'have', 'con', '##vo', '##lu', '##tion', '##al']
token_id :  [8107, 4719, 6125, 3522, 15756, 10047, 1011, 2811, 3512, 2836, 1012, 2007, 2784, 2031, 9530, 6767, 7630, 3508, 2389]


In [ ]:
original_text = tokenizer.decode(token_id)
original_text

'approaches achieved networks recent neural im - pressive performance. with deep have convolutional'

In [ ]:
#a function to tokenize all the rows in our dataset
max_text_length = 40 
max_label_length = 40 
def preprocess_function(sample):
    # Getting text and label
    text = sample["text"]
    label = sample["label"]
    # Tokenizing the text and label
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=max_text_length)
    outputs = tokenizer(label, padding="max_length", truncation=True, max_length=max_label_length)
    sample["input_ids"] = inputs.input_ids
    sample["attention_mask"] = inputs.attention_mask
    sample["decoder_input_ids"] = outputs.input_ids
    sample["decoder_attention_mask"] = outputs.attention_mask
    sample["labels"] = outputs.input_ids
    # The labels are used to calcuate the loss while training, and because we added padding to make all tokens to be of same size,
    # we also need to convert the padding number ( 0 ) to ( -100 ), so that we can tell huggingface that these number can be ignorned while calcuating loss. 
    # Why specifically -100 ? It's simply an arbitrary number, again so that huggingface can ignore this number while calcuating loss
    sample["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in sample["labels"]]

    return sample

In [ ]:
batch = 16
tok_dataset=dataset.map(preprocess_function,batch_size=batch,batched=True)

  0%|          | 0/251 [00:00<?, ?ba/s]

  0%|          | 0/625 [00:00<?, ?ba/s]

  0%|          | 0/251 [00:00<?, ?ba/s]

In [ ]:
tok_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

In [ ]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relations

In [ ]:
model#model architecture

In [ ]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size

In [ ]:
N_epochs = 1
args = Seq2SeqTrainingArguments(
    "Scambled Text",
    evaluation_strategy = "epoch",
    per_device_train_batch_size=batch,
    per_device_eval_batch_size=batch,
    num_train_epochs=N_epochs,
    fp16=True, # This will help in increasing the speed of training
)

In [ ]:
trainer = Seq2SeqTrainer(model = model,
                         args = args,
                         train_dataset = tok_dataset['train'],
                         eval_dataset = tok_dataset['val']
                        )

                                            
                         
                         

Using cuda_amp half precision backend


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: text, id. If text, id are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4001
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 251
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:533: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by 

Epoch,Training Loss,Validation Loss
1,No log,0.000174


The following columns in the evaluation set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: text, id. If text, id are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=251, training_loss=0.29378786125031126, metrics={'train_runtime': 89.6157, 'train_samples_per_second': 44.646, 'train_steps_per_second': 2.801, 'total_flos': 191752332501600.0, 'train_loss': 0.29378786125031126, 'epoch': 1.0})

In [ ]:
def generate_predictions(batch):

    # Tokenizing the test
    inputs = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=MAX_TEXT_LENGTH, return_tensors="pt")
    
    # Sending the tensors to GPU
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    # Generating the predicted tokens ids
    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # Converting the token ids to sentence
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["predictions"] = output_str

    return batch
    
    
    

In [ ]:
result = dataset['test'].map(predictions,batch_size = batch,batched=True)

  0%|          | 0/251 [00:00<?, ?ba/s]

TypeError: ignored

In [ ]:
test_text = test_df['text']
input = tokenizer(test_text,padding = 'max_length',max_length=max_text_length)
input_id = tokenizer.convert_tokens_to_ids(input)
attention_mask = input.attention_mask
output = model.generate(torch.tensor(input_id),attention_mask)
output = tokenizer.decode(output)


ValueError: ignored

In [ ]:
test_df.head(5)

,id,text,label
0,0,safely objects move. that system images detect...,"system Using approach, move. safely skip of th..."
1,1,We detectors popular influence of confidences ...,confidences popular different in detectors We ...
2,2,compact coding present supervised We a approac...,"We a supervised approach, compact present codi..."
3,3,study high-throughput vital of quantitative be...,is for and individuals study of of collective ...
4,4,on data sets. We evaluate method many challeng...,sets. challenging the data We method on many e...
